# Day 1

In [1]:
input <- readFile "./inputs/1/part1.txt"

In [5]:
parseInt :: String -> Integer
parseInt ('+':rest) = read rest
parseInt str = read str

In [6]:
let frequencies = map parseInt $ lines input

In [7]:
let resultingFrequency = sum frequencies

In [9]:
print resultingFrequency

500